In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [13]:
# 데이터 불러오기
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [14]:
# 컬럼 분석하기
df_train.columns

- Numerical 데이터 vs. Categorical 데이터 가 있음
- SalePrice에 미치는 영향의 정도에 따라 각 컬럼 분석
- 결과를 Conclusion 컬럼에 채워넣음

In [19]:
# SalePrice 컬럼 분석하기
# 분석의 목적이 되는 컬럼!
df_train['SalePrice'].describe()

- SalePrice의 개수, 평균, 분산, 최소, 구간값(25/50/75%), 최대

In [20]:
# histogram
sns.distplot(df_train['SalePrice']);

- SalePrice 값의 분포도를 막대그래프 형태로 알아봄
- 150000 ~ 200000 사이가 많은듯?

In [22]:
# skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

- 왜도(Skewness)
- 첨도(Kurtosis) : 피크 지점의 정도?

### Relationship with numerical variables

In [23]:
# Scatter plot grilivarea/saleprice
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000));

In [25]:
data

- SalePrice 와 GrlivArea 컬럼이 '선형관계' 임을 확인할 수 있다.

In [28]:
#scatter plot totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000));

- SalePrice와 TotalBsmSF 또한 마찬가지로 '선형관계'

### Relationship with categorical features

In [32]:
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

- SalePrice와 OverallQual...이건 무슨 관계지..?ㅆ?

In [36]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

- SalePrice와 YearBuilt
- 약한 관계이지만, 신식 부동산이 그나마 높은 가격인편


### 보다 객관적인 분석

### Correlation matrix (heatmap style)
- head + map : 데이터 값을 컬러로 변환시켜 열 분포 형태로 보여주어 시각화한 기법

In [43]:
# correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [49]:
# saleprice correlation matrix
k = 10 # 히트맵 변수 개수
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# corrmat : df_train의 컬럼끼리 상관관계 분석한 것
# corrmat.nlargest : corrmat 분석 중 상위 10개 컬럼 index(k=10으로 지정했을때) 추출

cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [45]:
np.corrcoef(df_train[cols].values.T)

- SalePriced와 다른 컬럼과의 상관관게
- 높은순으로 Overall, GrLivArea, GarageCars, GarageArea, TotalBsmtSF, 1stFlrSf...

In [50]:
# scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show();

### 누락된 데이터 분석하기

In [55]:
df_train.isnull()

In [62]:
df_train.isnull().count().sort_values(ascending=False)

In [59]:
df_train.isnull().sum().sort_values(ascending=False)

In [52]:
total = df_train.isnull().sum().sort_values(ascending=False)
total

In [53]:
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
percent

In [64]:
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

- 데이터가 15% 이상 누락된 경우, 해당 벼수 삭제
- Garage~~컬럼들의 누락 비율이 동일하므로, 삭제
- Bsmt~~도 마찬가지
- MasVnrArea, MasVnrType 삭제
- Electrical 변수 유지 (null값만 삭제, 1개)

In [65]:
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

### 이상치 분석
- 이상치냐 아니냐? 를 정의하는 임계값을 설정하기 위해 필요한 것 = 데이터 표준화
- 데이터 표준화 : 데이터 값 평균을 0, 표준편차가 1이 되도록 변환하는 것을 의미

In [74]:
# 데이터 표준화
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
saleprice_scaled

In [75]:
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

### 이변량 분석 (bivariate analysis)
- involving or depending on two variables

In [77]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [79]:
# deleting points
# id 가 1299, 524인 데이터 제거?
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [80]:
# bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [81]:
# 히스토그램 & 정규 확률도?

sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

- SalePrice가 양의 왜도 양상을 보임

In [82]:
df_train['SalePrice'] = np.log(df_train['SalePrice'])
df_train['SalePrice']

In [83]:
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

In [84]:
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [85]:
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [86]:
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [87]:
sns.distplot(df_train['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

In [89]:
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0 
df_train.loc[df_train['TotalBsmtSF'] > 0,'HasBsmt'] = 1

In [91]:
df_train.loc[df_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [92]:
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [93]:
plt.scatter(df_train['GrLivArea'], df_train['SalePrice']);

In [94]:
plt.scatter(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF'] > 0]['SalePrice']);

In [95]:
df_train = pd.get_dummies(df_train)